In [1]:
alias zkml source ~/.nvm/nvm.sh >/dev/null && nvm use 14.0.0 >/dev/null && yarn >/dev/null && ./zkml

In [4]:
zkml list_datasets

Available datasets:
[]


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
# Import Dataset from sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
from sklearn.datasets import load_iris # Load Iris Data

iris = load_iris() # Creating pd DataFrames

iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
target_df = pd.DataFrame(data= iris.target, columns= ['species'])

def converter(specie):
    if specie == 0:
        return 'setosa'
    elif specie == 1:
        return 'versicolor'
    else:
        return 'virginica'

target_df['species'] = target_df['species'].apply(converter)# Concatenate the DataFrames
iris_df = pd.concat([iris_df, target_df], axis= 1)

In [6]:
# Converting Objects to Numerical dtype
iris_df.drop('species', axis= 1, inplace= True)
target_df = pd.DataFrame(columns= ['species'], data= iris.target)
iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
y= iris_df['sepal length (cm)']

# Splitting the Dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.13, random_state= 101)

# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X_train, y_train)

# Making Predictions
lr.predict(X_test)
pred = lr.predict(X_test)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))

mse = mean_squared_error(y_test, pred)

Mean Squared Error: 0.1123029571353145


In [7]:
shuffle = lambda x: np.random.shuffle(x)

In [8]:
W = lr.coef_.reshape(-1, 1)
b = lr.intercept_.reshape(-1, 1)
X = X_test.values[:]
Yt_expected = y_test.values[:].reshape(-1, 1)
mse_target = 0.11232375

np.save('model/W.npy',W)
np.save('model/b.npy',b)
np.save('model/X.npy',X)
np.save('model/Y.npy',Yt_expected)

In [9]:
shuffle(W)
shuffle(b)
print(mean_squared_error(y_test, np.matmul(X_test, W.reshape(-1)) + b.reshape(-1)))

np.save('model_shuffled/W.npy',W)
np.save('model_shuffled/b.npy',b)
np.save('model_shuffled/X.npy',X)
np.save('model_shuffled/Y.npy',Yt_expected)

7.866197081771451


In [10]:
Y_expected = np.matmul(X_test, lr.coef_) + lr.intercept_

assert (Y_expected == lr.predict(X_test)).all()

In [12]:
from scripts import *
from copy import deepcopy
import json

data = dict(
    alpha_X = 0,
    beta_X = 8,

    alpha_W = -1,
    beta_W = 8,

    alpha_Y = 0,
    beta_Y = 8,

    alpha_Yt = 0,
    beta_Yt = 8,

    alpha_b = 0,
    beta_b = 8,

    alpha_R = -1,
    beta_R = 1,

    alpha_S = 0,
    beta_S = 0.4,

    m = 20,
    p = 4,
    n = 1,
    
    mse_target = mse_target
)

json.dump(data, open('./settings.json', 'w'), indent = 2)

In [13]:
zkml add_bounty --amount 11.1 --keyfile ./out --walletprivatekey ./keys/.private_key --model ./model --settings ./settings.json

Hashed inputs: 
14797455496207951391356508759149962584765968173479481191220882411966396840571n
Your Public Key: 
[
  9034578064572512214535301210539208408830664951051263007560019313942236166417n,
  4995903678092667726400702912124052816223616090560699902572601312415845443763n
]
Your Private Key: 
4405775790317722408439358757709180448963340536448745805265225550361191557706n
Current Balance
9051.244322055


In [14]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[
  {
    'publickey-1': '9034578064572512214535301210539208408830664951051263007560019313942236166417',
    'publickey-2': '4995903678092667726400702912124052816223616090560699902572601312415845443763',
    'MSE-Cap': '18406'
  }
]


In [15]:
zkml list_datasets

Available datasets:
[
  [
    'dataset',
    '14797455496207951391356508759149962584765968173479481191220882411966396840571'
  ]
]


In [16]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model_shuffled  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  7.866197081771451
... quantized  65536
Mean Squared Error simulated:  7.8678447170847842784
... quantized  65536
Circuit Outputs:
1288993
Proof took 19.115 s
ERROR: Invalid proof
An unexpected error occurred:

Error: Could not verify the proof
    at SimpleTaskDefinition.action (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/hardhat.config.js:245:26)
    at Environment._runTaskDefinition (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:217:14)
    at Environment.run (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/core/runtime-environment.ts:129:14)
    at main (/Users/liaopeiyuan/Documents/GitHub/zkml-core/eth/node_modules/hardhat/src/internal/cli/cli.ts:197:5)


In [17]:
zkml claim_bounty --payment 0x2546BcD3c84621e976D8185a91A922aE77ECEc30 --model ./model  --publickey ./keys/out_public.json --settings ./settings.json

Mean Squared Error actual:  0.11230295713531464
... quantized  18403
Mean Squared Error simulated:  0.11232375022889581895
... quantized  18406
Circuit Outputs:
18406
Proof took 19.779 s
INFO: OK!
Paying 0x2546BcD3c84621e976D8185a91A922aE77ECEc30
With balance
10523.3
Your Public Key: 
[
  14120564188937183059340295610980104431800856738831175893860994550799332578322n,
  12217560022818188977043687958118470319206917687952636978189460902086928554621n
]
Your Private Key: 
1954359726638143065852191709273647998442069713403378611944486203814191104207n
Current Balance
10534.4


In [18]:
zkml list_bounties --hash 14797455496207951391356508759149962584765968173479481191220882411966396840571

Available bounties on dataset: dataset
[]


In [19]:
zkml list_datasets

Available datasets:
[]
